In [1]:
from eig.monsters import Object, ObjectSetHypothesis, ObjectSetHypothesisSpace, UniformDistribution
import numpy as np
from math import log2
from itertools import permutations
from eig import compute_eig, Bayes, Context, entropy
from eig.monsters import Parser, Executor
from eig.monsters.program import ProgramSyntaxError

## Getting the answer to a question

### 1. Set up the environment

monsters:
- r = red
- b = blue
- p = purple

shapes: 
- 1 = square
- 2 = circle

legs (i.e., "size"): 
- 1 = 1 leg
- 2 = 2 legs
- 3 = 3 legs

In [2]:
monsters = [Object(object_label="r", shape=2, size=3, color="red"),
             Object(object_label="b", shape=1, size=2, color="blue"),
             Object(object_label="p", shape=2, size=1, color="purple")]

true_hypothesis = ObjectSetHypothesis(objects=monsters)

### 2. Parse question and execute it on the environment

In [3]:
# How many unique leg counts are there?
question = Parser.parse("(setSize (unique (map (lambda x0 (legs x0)) (set b r p))))")
executor = Executor(question)

answer = executor.execute(true_hypothesis)
print(answer)

3


In [4]:
# Does the purple monster have a circle head?
question = Parser.parse("(== (shape p) Circle)")
executor = Executor(question)

answer = executor.execute(true_hypothesis)
print(answer)

True


In [5]:
# Do the red and blue monsters have the same shaped head?
question = Parser.parse("(== (shape r) (shape b))")
executor = Executor(question)

answer = executor.execute(true_hypothesis)
print(answer)

False


## Computing a question's informativeness (EIG)

### 1. Set up prior belief state (before hint)

In [6]:
hypotheses = ObjectSetHypothesisSpace(object_colors=["red", "blue", "purple"], object_sizes=[1, 2, 3], object_shapes=[1, 2])
prior = UniformDistribution()  
belief = Bayes(hypotheses, prior) # belief over hypotheses

### 1. Update beliefs to account for hint

Hint takes the following form: [[red_head, red_legs], [blue_head, blue_legs], [purple_head, purple_legs]]

For each entry, -1 indicates that the feature is hidden.

So, for example, if the hint is [[-1, 2], [1, 3], [-1, -1]], this means:
- the red monster has a hidden head and two legs
- the blue monster has a square head and three legs
- the purple monster has a hidden head and hidden legs

As above, 1 = square, 2 = circle, 1 = 1 leg, 2 = 2 legs, 3 = 3 legs

In [11]:
context = Context(hypotheses, belief)   
context.observe(np.array([[1, 2], [-1, 3], [-1, 2]]))

# can also get entropy of current belief (how much uncertainty is there?)
entropy(context.belief.belief)

2.0

### 3. Parse question and compute EIG

In [32]:
# Does the blue monster have a Square head?
question = Parser.parse("(== (shape b) Square)")
executor = Executor(question)

eig = compute_eig(executor, context)
print(eig)

1.0


In [33]:
# How many monsters have a square head?
question = Parser.parse("(++ (map (lambda x0 (== (shape x0) Square)) (set b r p)))")
executor = Executor(question)

eig = compute_eig(executor, context)
print(eig)

1.5


In [34]:
# How many legs do all the monsters have combined together?
question = Parser.parse("(++ (map (lambda x0 (legs x0)) (set b r p)))")
executor = Executor(question)

eig = compute_eig(executor, context)
print(eig)

-0.0
